In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [3]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.000651                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.001014
0.000750      2.0  0.000684                  2.0  0.001039
0.001052      3.0  0.000755                  3.0  0.001111
0.001476      4.0  0.000828                  4.0  0.001177
0.002070      5.0  0.000904                  5.0  0.001245
0.002904      6.0  0.000982                  6.0  0.001314
0.004074      7.0  0.001061                  7.0  0.001385
0.005714      8.0  0.001143                  8.0  0.001457
0.008015      9.0  0.001226                  9.0  0.001531
0.011243     10.0  0.001313                 10.0  0.001608
0.015771     11.0  0.001399                 11.0  0.001687
0.022122     12.0  0.001484                 12.0  0.001767
0.031031     13.0  0.001564                 13.0  0.001849
0.043528     14.0  0.001672                 14.0  0.001955
0.061057     15.0  0.002116                 15.0  0.002362
0.085645     16.0  0.003169                 16.0  0.003306
0.120136     17.0  0.004719                 17.0  0.004676
0.168516     18.0  0.006666                 18.0  0.006390
0.236378     19.0  0.009069                 19.0  0.008506
0.331549     20.0  0.011994                 20.0  0.011092
0.465100     21.0  0.015508                 21.0  0.014255
0.652400     22.0  0.019213                 22.0  0.017724
0.915100     23.0  0.020144                 23.0  0.018957
1.283650     24.0  0.018033                 24.0  0.017585
1.800600     25.0  0.015114                 25.0  0.015334
2.525700     26.0  0.011919                 26.0  0.012593
3.542800     27.0  0.009230                 27.0  0.010045
4.969550     28.0  0.007164                 28.0  0.007869
6.970850     29.0  0.005599                 29.0  0.006039
9.778100     30.0  0.004414                 30.0  0.004538
13.715850    31.0  0.003494                 31.0  0.003332
19.239350    32.0  0.002747                 32.0  0.002379
26.987250    33.0  0.002107                 33.0  0.001623
37.855300    34.0  0.001515                 34.0  0.001007
53.100050    35.0  0.000776                 35.0  0.000381
73.887500    36.0  0.000047                 36.0 -0.000152
97.662500    37.0 -0.000298                 37.0 -0.000369
121.437500   38.0 -0.000142                 38.0 -0.000230
145.212500   39.0  0.000414                 39.0  0.000190
168.987500   40.0  0.001215                 40.0  0.000792
192.762500   41.0  0.003029                 41.0  0.002161
216.537500   42.0  0.009242                 42.0  0.007153
240.312500   43.0  0.020643                 43.0  0.017319
264.087500   44.0  0.036832                 44.0  0.032434
287.862500   45.0  0.056430                 45.0  0.050245
311.637500   46.0  0.078502                 46.0  0.070939
335.412500   47.0  0.102735                 47.0  0.096205
359.187500   48.0  0.127589                 48.0  0.120380
382.962500   49.0  0.146890                 49.0  0.134631
406.737500   50.0  0.169850                 50.0  0.153448
430.512500   51.0  0.195782                 51.0  0.180679
454.287500   52.0  0.224281                 52.0  0.213162
478.062500   53.0  0.254887                 53.0  0.246007
501.837500   54.0  0.286508                 54.0  0.277070
525.612500   55.0  0.318883                 55.0  0.309484
549.387500   56.0  0.350704                 56.0  0.343524
573.162500   57.0  0.381219                 57.0  0.379118
596.937500   58.0  0.409825                 58.0  0.413989
620.712500   59.0  0.435420                 59.0  0.444769
644.487500   60.0  0.457963                 60.0  0.471492
668.262500   61.0  0.476519                 61.0  0.493725
692.037500   62.0  0.488763                 62.0  0.510076
715.812500   63.0  0.492375                 63.0  0.519452
739.587500   64.0  0.513244                 64.0  0.549936
76

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -30.008535  0.000000e+00 -30.008535        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -30.074835   0.000002   
6.244000e-04 2     -30.008535  2.089206e-07 -30.008535 -30.074835   0.000002   
8.759000e-04 3     -30.008535  2.939809e-07 -30.008534 -30.074835   0.000002   
1.228600e-03 4     -30.008535  4.159465e-07 -30.008534 -30.074835   0.000002   
1.723400e-03 5     -30.008535  5.909458e-07 -30.008534 -30.074835   0.000002   
2.417400e-03 6     -30.008535  8.419739e-07 -30.008534 -30.074835   0.000003   
3.390900e-03 7     -30.008535  1.202074e-06 -30.008534 -30.074835   0.000003   
4.756500e-03 8     -30.008536  1.718607e-06 -30.008534 -30.074835   0.000003   
6.672000e-03 9     -30.008536  2.459442e-06 -30.008534 -30.074836   0.000004   
9.358900e-03 10    -30.008537  3.521943e-06 -30.008533 -30.074836   0.000005   
1.312780e-02 11    -30.008538  5.045663e-06 -30.008533 -30.074836   0.000006   
1.841450e-02 12    -30.008539  7.230721e-06 -30.008532 -30.074837   0.000008   
2.583020e-02 13    -30.008541  1.036082e-05 -30.008531 -30.074838   0.000010   
3.623230e-02 14    -30.008544  1.483699e-05 -30.008529 -30.074840   0.000014   
5.082340e-02 15    -30.008547  2.127731e-05 -30.008526 -30.074841   0.000019   
7.129060e-02 16    -30.008552  3.120260e-05 -30.008521 -30.074844   0.000027   
1.000000e-01 17    -30.008558  4.795412e-05 -30.008510 -30.074846   0.000041   
1.402710e-01 18    -30.008564  7.698137e-05 -30.008487 -30.074848   0.000065   
1.967600e-01 19    -30.008570  1.269304e-04 -30.008443 -30.074846   0.000106   
2.759970e-01 20    -30.008570  2.121522e-04 -30.008358 -30.074836   0.000176   
3.871000e-01 21    -30.008556  3.561026e-04 -30.008200 -30.074810   0.000295   
5.431000e-01 22    -30.008510  5.967831e-04 -30.007913 -30.074749   0.000498   
7.617000e-01 23    -30.008401  9.854719e-04 -30.007416 -30.074626   0.000834   
1.068500e+00 24    -30.008217  1.533809e-03 -30.006684 -30.074428   0.001325   
1.498800e+00 25    -30.007974  2.209305e-03 -30.005764 -30.074168   0.001962   
2.102400e+00 26    -30.007677  2.992967e-03 -30.004684 -30.073851   0.002741   
2.949000e+00 27    -30.007344  3.855245e-03 -30.003488 -30.073491   0.003644   
4.136600e+00 28    -30.006982  4.791759e-03 -30.002190 -30.073097   0.004663   
5.802500e+00 29    -30.006594  5.818132e-03 -30.000776 -30.072674   0.005793   
8.139200e+00 30    -30.006191  6.964943e-03 -29.999226 -30.072237   0.007028   
1.141700e+01 31    -30.005794  8.281110e-03 -29.997513 -30.071816   0.008369   
1.601470e+01 32    -30.005443  9.833567e-03 -29.995610 -30.071466   0.009834   
2.246400e+01 33    -30.005215  1.170373e-02 -29.993511 -30.071278   0.011464   
3.151050e+01 34    -30.005236  1.398208e-02 -29.991254 -30.071395   0.013320   
4.420010e+01 35    -30.005722  1.674557e-02 -29.988977 -30.072028   0.015466   
6.200000e+01 36    -30.007157  1.981723e-02 -29.987340 -30.073568   0.017811   
8.577500e+01 37    -30.010013  2.280501e-02 -29.987208 -30.076303   0.020117   
1.095500e+02 38    -30.013410  2.536158e-02 -29.988049 -30.079346   0.022120   
1.333250e+02 39    -30.017062  2.861246e-02 -29.988450 -30.082520   0.024645   
1.571000e+02 40    -30.020835  3.355133e-02 -29.987284 -30.085769   0.028431   
1.808750e+02 41    -30.024931  4.107038e-02 -29.983861 -30.089297   0.034189   
2.046500e+02 42    -30.030588  5.525656e-02 -29.975331 -30.094242   0.045220   
2.284250e+02 43    -30.042263  9.295970e-02 -29.949304 -30.104712   0.075838   
2.522000e+02 44    -30.063122  1.719567e-01 -29.891165 -30.123785   0.143692   
2.759750e+02 45    -30.096632  3.091975e-01 -29.787435 -30.154893   0.266155   
2.997500e+02 46    -30.146348  5.178389e-01 -29.628509 -30.201613   0.454395   
3.235250e+02 47    -30.215

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')